<a href="https://colab.research.google.com/github/codeREXus/langchain-learnings/blob/main/Langchain5_chromaDB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install chromadb==1.0.12
!pip install sentence-transformers==4.1.0

In [25]:
import chromadb
from chromadb.utils import embedding_functions

ef= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name ="all-MiniLm-L6-V2"
)

In [34]:
client=chromadb.Client()
collection_name= 'my_grocery'

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [42]:
def main():
  try:
    global collection
    collection = client.create_collection(
    name=collection_name,
    metadata={"description": "A collection for storing grocery data"},
    configuration={
        "hnsw": {"space": "cosine"},
        "embedding_function": ef,
        'ef_search':60,
        'ef_construction':120
        }
    )
    print(f"Collection created: {collection.name}")


    texts = [
    'fresh red apples',
    'organic bananas',
    'ripe mangoes',
    'whole wheat bread',
    'farm-fresh eggs',
    'natural yogurt',
    'frozen vegetables',
    'grass-fed beef',
    'free-range chicken',
    'fresh salmon fillet',
    'aromatic coffee beans',
    'pure honey',
    'golden apple',
    'red fruit'
    ]


    ids = [f"food_{index + 1}" for index, _ in enumerate(texts)]

    collection.add(
    documents=texts,
    metadatas=[{'source':'grocery', 'catgory':'food'} for _ in texts],
    ids= ids
    )
    all_item= collection.get()


    sm_search(collection, all_item)
  except Exception as e:
    print(f"Error {e}")

In [79]:
def sm_search(collection, all_item):
  try:
    global query_term
    query_term=['carrot','potato']
    if isinstance(query_term, str):
        query_term = [query_term]

    results = collection.query(
    query_texts=query_term,
    n_results=3
    )
    print(f"Query results for '{query_term}':")
    print(results['distances'])

  #   if not results or not results['ids'] or len(results['ids'][0]) == 0:
  #     print(f'No documents found similar to "{query_term}"')
  #     return

  #   print(f'Top 3 similar documents to "{query_term}":')

  #   for q in range(len(query_term)):
  #     print(f'Top 3 similar documents to "{query_term[q]}":')
  #     # Access the nested arrays in 'results["ids"]' and 'results["distances"]'
  #     for i in range(min(3, len(results['ids'][q]))):
  #         doc_id = results['ids'][q][i]  # Get ID from 'ids' array
  #         score = results['distances'][q][i]  # Get score from 'distances' array
  #         # Retrieve text data from the results
  #         text = results['documents'][q][i]
  #         if not text:
  #             print(f' - ID: {doc_id}, Text: "Text not available", Score: {score:.4f}')
  #         else:
  #             print(f' - ID: {doc_id}, Text: "{text}", Score: {score:.4f}')
  # except Exception as error:
  #     print(f"Error in similarity search: {error}")
  # except Exception as error :
  #   print(f"Error in similarity search: {error}")
  except Exception as e:
    print(f"Error {e}")

In [80]:
client.delete_collection('my_grocery')

In [81]:
if __name__ == "__main__":
  main()

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Collection created: my_grocery
Query results for '['carrot', 'potato']':
[[0.4914088845252991, 0.5408385396003723, 0.5603157877922058], [0.5108054876327515, 0.560166597366333, 0.5936388373374939]]
